# Projeto: Impacto do Airbnb em Fatores Urbanos em Portugal
Este notebook faz parte da Fase 1 do projeto da unidade curricular.

## Objetivos:
- Integrar dados de diferentes fontes (Airbnb, INE, etc.)
- Analisar a relação entre densidade de alojamentos locais, população e rendas
- Implementar técnicas de data cleaning, schema integration e identity resolution


In [16]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Leitura dos datasets
listings = pd.read_csv(r'dados\portugal_listings.csv')
rendas = pd.read_json(r'dados\rendasm2.json')
rendas = pd.DataFrame(rendas['Dados'][0]['2023'])
densidade_pop = pd.read_json(r'dados\densidadePopulacional.json')
densidade_pop = pd.DataFrame(densidade_pop['Dados'][0]['2022'])
densidade_aloj = pd.read_json(r'dados\densidadealojamentosm2.json')
densidade_aloj = pd.DataFrame(densidade_aloj['Dados'][0]['2021'])

## 3. Data Profiling

- Número de entradas
- Tipos de dados
- Percentagem de valores nulos
- Cardinalidade de colunas relevantes


In [22]:
listings.info()
listings.describe()
listings.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135536 entries, 0 to 135535
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Price        135236 non-null  float64
 1   District     135536 non-null  object 
 2   City         135536 non-null  object 
 3   Town         135534 non-null  object 
 4   Unnamed: 4   0 non-null       float64
 5   Unnamed: 5   0 non-null       float64
 6   Unnamed: 6   0 non-null       float64
 7   Unnamed: 7   0 non-null       float64
 8   Unnamed: 8   0 non-null       float64
 9   Unnamed: 9   0 non-null       float64
 10  Unnamed: 10  0 non-null       float64
 11  Unnamed: 11  0 non-null       float64
 12  Unnamed: 12  0 non-null       float64
 13  Unnamed: 13  0 non-null       float64
dtypes: float64(11), object(3)
memory usage: 14.5+ MB


Price             300
District            0
City                0
Town                2
Unnamed: 4     135536
Unnamed: 5     135536
Unnamed: 6     135536
Unnamed: 7     135536
Unnamed: 8     135536
Unnamed: 9     135536
Unnamed: 10    135536
Unnamed: 11    135536
Unnamed: 12    135536
Unnamed: 13    135536
dtype: int64

portugal_listings.csv

| Coluna                  | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada               |
|-------------------------|--------------|-------------------|----------------|---------|-----------------------------|
| `Price`                 | float64      | 135236            | 300            | 0.22%   | Remover nulos ou imputar    |
| `District`              | object       | 135536            | 0              | 0.00%   | Manter                      |
| `City`                  | object       | 135536            | 0              | 0.00%   | Manter                      |
| `Town`                  | object       | 135534            | 2              | 0.001%  | Imputar com `City` ou remover |
| `Unnamed: 4` a `Unnamed: 13` | float64 | 0              | 135536         | 100%    | Remover todas               |


In [23]:
rendas.info()
rendas.describe()
rendas.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   geocod           638 non-null    object
 1   geodsg           638 non-null    object
 2   ind_string       638 non-null    object
 3   valor            431 non-null    object
 4   sinal_conv       207 non-null    object
 5   sinal_conv_desc  207 non-null    object
dtypes: object(6)
memory usage: 30.0+ KB


geocod               0
geodsg               0
ind_string           0
valor              207
sinal_conv         431
sinal_conv_desc    431
dtype: int64

rendasm2.json

| Coluna            | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada                                 |
|-------------------|--------------|-------------------|----------------|---------|-----------------------------------------------|
| `geocod`          | object       | 638               | 0              | 0.00%   | Manter                                        |
| `geodsg`          | object       | 638               | 0              | 0.00%   | Usar como chave para junção                   |
| `ind_string`      | object       | 638               | 0              | 0.00%   | Ignorar                                       |
| `valor`           | object       | 431               | 207            | 32.45%  | Eliminar ou ignorar entradas nulas            |
| `sinal_conv`      | object       | 207               | 431            | 67.55%  | Apenas explicativo (motivo da ausência de valor) |
| `sinal_conv_desc` | object       | 207               | 431            | 67.55%  | Ignorar ou mover para anexo                   |


In [19]:
densidade_pop.info()
densidade_pop.describe()
densidade_pop.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   geocod      344 non-null    object
 1   geodsg      344 non-null    object
 2   ind_string  344 non-null    object
 3   valor       344 non-null    object
dtypes: object(4)
memory usage: 10.9+ KB


geocod        0
geodsg        0
ind_string    0
valor         0
dtype: int64

densidadePopulacional.json

| Coluna       | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada                     |
|--------------|--------------|-------------------|----------------|---------|-----------------------------------|
| `geocod`     | object       | 344               | 0              | 0.00%   | Manter                            |
| `geodsg`     | object       | 344               | 0              | 0.00%   | Usar como chave                   |
| `ind_string` | object       | 344               | 0              | 0.00%   | Ignorar                           |
| `valor`      | object       | 344               | 0              | 0.00%   | Converter para float              |


In [20]:
densidade_aloj.info()
densidade_aloj.describe()
densidade_aloj.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3439 entries, 0 to 3438
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   geocod      3439 non-null   object
 1   geodsg      3439 non-null   object
 2   ind_string  3439 non-null   object
 3   valor       3439 non-null   object
dtypes: object(4)
memory usage: 107.6+ KB


geocod        0
geodsg        0
ind_string    0
valor         0
dtype: int64

densidadelojamentosm2.json

| Coluna       | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada                     |
|--------------|--------------|-------------------|----------------|---------|-----------------------------------|
| `geocod`     | object       | 344               | 0              | 0.00%   | Manter                            |
| `geodsg`     | object       | 344               | 0              | 0.00%   | Usar como chave                   |
| `ind_string` | object       | 344               | 0              | 0.00%   | Ignorar                           |
| `valor`      | object       | 344               | 0              | 0.00%   | Converter para float              |


### Conclusões do Data Profiling

Após a leitura e análise dos datasets, foi possível identificar a seguinte situação:

- Os dados extraídos dos ficheiros `.json` e `.csv` foram corretamente convertidos em tabelas pandas a partir da chave `"Dados"`.
- As colunas `valor`, apesar de estarem no formato `object`, podem ser convertidas para `float` após a substituição da vírgula pelo ponto decimal.
- No dataset das rendas (`rendasm2.json`), foi detetada uma percentagem significativa de valores nulos (~32%) na coluna `valor`, que deverão ser removidos ou imputados.
- Todos os datasets possuem a coluna `geodsg`, que será usada como **chave principal de junção**.
- A coluna `geocod` foi mantida por agora, uma vez que poderá ser útil para cruzamentos futuros com dados geográficos oficiais.
- Os campos como `ind_string`, `sinal_conv` e `sinal_conv_desc` foram considerados auxiliares e não serão usados na análise principal.

**Próximos passos:**
- Conversão das colunas `valor` para `float`.
- Remoção dos registos com valores nulos nas variáveis principais.
- Normalização de nomes para futura integração entre datasets.


In [73]:
import pandas as pd

# Load the CSV file
file_path = 'dados/portugal_listings.csv'  # Replace with the actual file path
df = pd.read_csv(file_path)

# Print the column names to verify
print("Columns in the DataFrame:", df.columns)

# Rename columns (if necessary)
df = df.rename(columns={
    'Price': 'Price',
    'District': 'District',
    'City': 'City',
    'Town': 'Town'
    # Add more columns as needed
})

# Convert Price to numeric (if it's not already)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Handle missing values (e.g., replace NaN with a default value or drop rows)
df['Price'] = df['Price'].fillna(0)  # Replace NaN with 0, or use another strategy

# Filter out rows with missing or non-applicable data
df_filtered = df[df['Price'].notna()]

# Display the cleaned DataFrame
print(df_filtered.head())


Columns in the DataFrame: Index(['Price', 'District', 'City', 'Town', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')
      Price   District                  City  \
0  780000.0  Vila Real              Valpaços   
1  223000.0       Faro  São Brás de Alportel   
2  228000.0       Faro  São Brás de Alportel   
3  250000.0       Faro  São Brás de Alportel   
4  250000.0       Faro  São Brás de Alportel   

                               Town  Unnamed: 4  Unnamed: 5  Unnamed: 6  \
0  Carrazedo de Montenegro e Curros         NaN         NaN         NaN   
1              São Brás de Alportel         NaN         NaN         NaN   
2              São Brás de Alportel         NaN         NaN         NaN   
3              São Brás de Alportel         NaN         NaN         NaN   
4              São Brás de Alportel         NaN         NaN         NaN   

   Unname

In [7]:
import xml.etree.ElementTree as ET

tree = ET.parse('concelhos.up.xml')  # Parse XML file
root = tree.getroot()  # Get root element

objects_data = []
for object_element in root.findall("Object"):
    object_dict = {}
    for property_element in object_element.findall("Property"):
        name = property_element.get("Name")
        value = property_element.text
        object_dict[name] = value
    objects_data.append(object_dict)

df = pd.DataFrame(objects_data)
    
print(df)

               Concelho                       Address    Latitude   Longitude
0              Abrantes            Abrantes, Portugal  39.4630563  -8.1995808
1                Águeda              Águeda, Portugal  40.5754246  -8.4464368
2       Aguiar da Beira     Aguiar da Beira, Portugal   40.816453  -7.5454104
3             Alandroal           Alandroal, Portugal  38.7021331  -7.4036488
4    Albergaria-a-Velha  Albergaria-a-Velha, Portugal  40.6894236  -8.4796655
..                  ...                           ...         ...         ...
300             Vimioso             Vimioso, Portugal  41.5844451  -6.5291211
301             Vinhais             Vinhais, Portugal  41.8307225  -7.0093005
302               Viseu               Viseu, Portugal  40.6565861  -7.9124712
303              Vizela              Vizela, Portugal  41.3764108  -8.3098348
304             Vouzela             Vouzela, Portugal  40.7231807  -8.1120232

[305 rows x 4 columns]


In [63]:
import json
import pandas as pd

# Load the JSON file
file_path = 'dados/densidadealojamentosm2.json'  # Replace with the actual file path
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract the 2021 data
dados_2021 = data[0]['Dados']['2021']

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(dados_2021)

# Print the column names to verify
print("Columns in the DataFrame:", df.columns)

# Rename columns
df = df.rename(columns={
    'geocod': 'RegionCode',
    'geodsg': 'RegionName',
    'ind_string': 'DensityString',
    'valor': 'DensityValue'  # Ensure this matches the actual column name
})

# Convert DensityValue to numeric
df['DensityValue'] = pd.to_numeric(df['DensityValue'], errors='coerce')

# Handle missing values (e.g., replace "-" with NaN)
df['DensityValue'] = df['DensityValue'].replace('-', None)

# Filter out rows with missing or non-applicable data
df_filtered = df[df['DensityValue'].notna()]

# Display the cleaned DataFrame
print(df_filtered.head())

Columns in the DataFrame: Index(['geocod', 'geodsg', 'ind_string', 'valor'], dtype='object')
  RegionCode                                         RegionName DensityString  \
0     070513                                Torre de Coelheiros           2,1   
1     070601                                            Cabrela           2,2   
2     031003                                     Campo do Gerês           2,3   
3     120302                                       Aldeia Velha           2,4   
4     070525  União das freguesias de Nossa Senhora da Toure...           2,5   

   DensityValue  
0           2.1  
1           2.2  
2           2.3  
3           2.4  
4           2.5  


In [57]:
import json
import pandas as pd

# Load the JSON file
file_path = 'dados/densidadePopulacional.json'  # Replace with the actual file path
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract the 2022 data
dados_2022 = data[0]['Dados']['2022']

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(dados_2022)

# Print the column names to verify
print("Columns in the DataFrame:", df.columns)

# Rename columns
df = df.rename(columns={
    'geocod': 'RegionCode',
    'geodsg': 'RegionName',
    'ind_string': 'DensityString',
    'valor': 'DensityValue'  # Ensure this matches the actual column name
})

# Convert DensityValue to numeric
df['DensityValue'] = pd.to_numeric(df['DensityValue'], errors='coerce')

# Handle missing values (e.g., replace "-" with NaN)
df['DensityValue'] = df['DensityValue'].replace('-', None)

# Filter out rows with missing or non-applicable data
df_filtered = df[df['DensityValue'].notna()]

# Display the cleaned DataFrame
print(df_filtered.head())

Columns in the DataFrame: Index(['geocod', 'geodsg', 'ind_string', 'valor'], dtype='object')
  RegionCode     RegionName DensityString  DensityValue
0    1500802       Alcoutim           4,3           4.3
1    1840209        Mértola           4,8           4.8
2    16H0505  Idanha-a-Nova           5,9           5.9
3    1861203           Avis           6,2           6.2
4    1861211       Monforte           7,1           7.1


In [53]:
import json
import pandas as pd

# Load the JSON file
file_path = 'dados/rendasm2.json'  # Replace with the actual file path
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract the 2023 data
dados_2023 = data[0]['Dados']['2023']

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(dados_2023)

# Print the column names to verify
print("Columns in the DataFrame:", df.columns)

# Rename columns
df = df.rename(columns={
    'geocod': 'RegionCode',
    'geodsg': 'RegionName',
    'ind_string': 'RentString',
    'valor': 'RentValue',  # Ensure this matches the actual column name
    'sinal_conv': 'SignalCode',
    'sinal_conv_desc': 'SignalDescription'
})

# Convert RentValue to numeric
df['RentValue'] = pd.to_numeric(df['RentValue'], errors='coerce')

# Handle missing values (e.g., replace "-" with NaN)
df['RentValue'] = df['RentValue'].replace('-', None)

# Filter out rows with missing or non-applicable data
df_filtered = df[df['RentValue'].notna()]

# Display the cleaned DataFrame
print(df_filtered.head())

Columns in the DataFrame: Index(['geocod', 'geodsg', 'ind_string', 'valor', 'sinal_conv',
       'sinal_conv_desc'],
      dtype='object')
  RegionCode            RegionName RentString  RentValue SignalCode  \
0    11D0914  Vila Nova de Foz Côa       2,08       2.08        NaN   
1    11E0410             Vila Flor       2,29       2.29        NaN   
2    1960913              Trancoso       2,41       2.41        NaN   
3    1960501              Belmonte       2,61       2.61        NaN   
4    11C1302                 Baião       2,63       2.63        NaN   

  SignalDescription  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
